# RNN without Transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/MACHINE PROJECT/rs2_augmented.csv")

In [4]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["sentence1_clean"].tolist() + df["sentence2_clean"].tolist())

# Convert to sequences
s1_seq = tokenizer.texts_to_sequences(df["sentence1_clean"])
s2_seq = tokenizer.texts_to_sequences(df["sentence2_clean"])

# Pad sequences
max_len = 50
X1 = pad_sequences(s1_seq, maxlen=max_len, padding='post')
X2 = pad_sequences(s2_seq, maxlen=max_len, padding='post')
y = df["score"].values


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Lambda, Dense, Dropout, concatenate
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

# Train-validation split
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
    X1, X2, y, test_size=0.2, random_state=42
)

# Define parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

# Inputs
input1 = Input(shape=(max_len,))
input2 = Input(shape=(max_len,))

# Shared embedding + RNN encoder
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)

embedded1 = embedding_layer(input1)
embedded2 = embedding_layer(input2)

shared_lstm = Bidirectional(LSTM(64, return_sequences=False))

encoded1 = shared_lstm(embedded1)
encoded2 = shared_lstm(embedded2)

# Feature interaction
abs_diff = Lambda(lambda x: tf.abs(x[0] - x[1]))([encoded1, encoded2])
mult = Lambda(lambda x: x[0] * x[1])([encoded1, encoded2])
merged = concatenate([encoded1, encoded2, abs_diff, mult])

# Dense layers
dense = Dense(128, activation='relu')(merged)
drop = Dropout(0.3)(dense)
output = Dense(1, activation='linear')(drop)

model = Model(inputs=[input1, input2], outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# Train
history = model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_val, X2_val], y_val),
    epochs=10, batch_size=32
)
y_pred = model.predict([X1_val, X2_val]).flatten()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 1002s 83ms/step - loss: 1.6080 - mae: 1.0071 - val_loss: 0.7105 - val_mae: 0.6183
Epoch 2/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 998s 83ms/step - loss: 0.5753 - mae: 0.5578 - val_loss: 0.5349 - val_mae: 0.5188
Epoch 3/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 998s 83ms/step - loss: 0.3876 - mae: 0.4459 - val_loss: 0.4404 - val_mae: 0.4466
Epoch 4/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 1000s 83ms/step - loss: 0.3016 - mae: 0.3883 - val_loss: 0.4040 - val_mae: 0.4250
Epoch 5/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 998s 83ms/step - loss: 0.2500 - mae: 0.3503 - val_loss: 0.3635 - val_mae: 0.3911
Epoch 6/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 998s 83ms/step - loss: 0.2171 - mae: 0.3242 - val_loss: 0.3364 - val_mae: 0.3681
Epoch 7/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 999s 83ms/step - loss: 0.1965 - mae: 0.3069 - val_loss: 0.3262 - val_mae: 0.3594
Epoch 8/10
12071/12071 ━━━━━━━━━━━━━━━━━━━━ 998s 83ms/step - loss: 0.1781 - mae: 0.2912 - val_loss: 0.3158 - val_mae: 0.350

In [1]:
from scipy.stats import pearsonr, spearmanr
import matplotib.pyplot as plt
# Pearson Correlation
pearson_corr, _ = pearsonr(y_val, y_pred)

# Spearman Correlation
spearman_corr, _ = spearmanr(y_val, y_pred)

print(f"📈 Pearson Correlation:  {pearson_corr:.4f}")
print(f"📊 Spearman Correlation: {spearman_corr:.4f}")


# Plot loss and MAE
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()

# MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('MAE over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()




NameError: name 'y_val' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt

# Helper function
def evaluate(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE": mean_absolute_percentage_error(y_true, y_pred),
        "R²": r2_score(y_true, y_pred)
    }


print(evaluate(y_val, y_pred))


{'MSE': 0.004233996849507093, 'MAE': 0.053695619106292725, 'MAPE': 0.010739123448729515, 'R²': 0.0}
